In [ ]:
import numpy as np
from Bio import SeqIO
from collections import Counter

# Load protein sequences from FASTA files
def load_sequences(fasta_file):
    sequences = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(str(record.seq))
    return sequences

# Define the feature computation function
def compute_manual_features(sequence):
    # Amino Acid Composition (ACC)
    def compute_acc(sequence):
        amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
        counts = Counter(sequence)
        acc = np.array([counts[aa] / len(sequence) for aa in amino_acids])
        return acc

    # Composition of K-Spaced Amino Acid Pairs (CKSAAP)
    def compute_cksaap(sequence, k=2):
        amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
        pairs = [a1 + a2 for a1 in amino_acids for a2 in amino_acids]
        cksaap = []
        for i in range(k + 1):
            counts = Counter([sequence[j] + sequence[j + i + 1] for j in range(len(sequence) - i - 1)])
            cksaap.extend([counts[pair] / (len(sequence) - i - 1) for pair in pairs])
        return np.array(cksaap)

    # Dipeptide Deviation from Expected Mean (DDE)
    def compute_dde(sequence):
        amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
        counts = Counter(sequence)
        total_pairs = len(sequence) - 1
        dde = []
        for aa1 in amino_acids:
            for aa2 in amino_acids:
                observed = sequence.count(aa1 + aa2)
                expected = (counts[aa1] / len(sequence)) * (counts[aa2] / len(sequence)) * total_pairs
                dde.append((observed - expected) / total_pairs)
        return np.array(dde)

    # Grouped Tripeptide Composition (GTPC)
    def compute_gtpc(sequence):
        groups = {
            'A': 'GAVLMI',  # Non-polar
            'B': 'FWY',     # Aromatic
            'C': 'KRH',     # Positively charged
            'D': 'DE',      # Negatively charged
            'E': 'STNQ',    # Polar, uncharged
        }
        group_keys = list(groups.keys())
        group_counts = {g: 0 for g in group_keys}
        for i in range(len(sequence) - 2):
            tri = sequence[i:i + 3]
            for g in group_keys:
                if all(char in groups[g] for char in tri):
                    group_counts[g] += 1
        total_tris = len(sequence) - 2
        gtpc = np.array([group_counts[g] / total_tris for g in group_keys])
        return gtpc

    # Concatenate all features
    # acc = compute_acc(sequence)
    # cksaap = compute_cksaap(sequence)
    # dde = compute_dde(sequence)
    gtpc = compute_gtpc(sequence)
    return np.concatenate([gtpc])

# Load sequences from files
positive_sequences = load_sequences("/content/drive/MyDrive/clustered_positive_0.4.fasta")
negative_sequences = load_sequences("/content/drive/MyDrive/clustered_negative_0.4.fasta")

# Assign labels: 1 for positive, 0 for negative
sequences = positive_sequences + negative_sequences
labels = [1] * len(positive_sequences) + [0] * len(negative_sequences)

# Compute manual features for all sequences
X = np.array([compute_manual_features(seq) for seq in sequences])
y = np.array(labels)

# Save features and labels to .npy files
np.save('/content/drive/MyDrive/manu_features_gtpc.npy', X)
np.save('/content/drive/MyDrive/manu_labels_gtpc.npy', y)

print("Feature extraction completed and saved to 'features.npy' and 'labels.npy'.")
